<a href="https://colab.research.google.com/github/annykay/scientificComputations/blob/main/SciComp2021_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scientific Computing 2021: Homework Assignment 1 
Due Monday October 11, 2021, 23:59

### Problem 1 (2 points)
Under assumptions of Amdahl's law, suppose that 75% of a program are perfectly parallelizable, and the rest is not parallelizable. 
1. What is the maximum speedup achievable by parallelization? 
2. Suppose that we have obtained speedup 2 (by using a suitable number of processes). What is the efficiency of this parallelization? 

### Problem 2 (2 points)
Write a Python or C/C++ program that uses **MPI reduce** to find the largest file in terms of the  number of lines among all .txt files in the working directory (only .txt files should be examined). The program must be callable in the form `mpirun -np <N> python linecount.py` (in the case of the Python version) or `mpirun -np <N> linecount.exe` (the C/C++ version), where `<N>` is the user-defined number of processes. The program is expected to first somehow distribute the files found in the current directory to the processes, then each process should count the lines in the files assigned to it, and finally the result should be MPI-reduced and printed out. The program only needs to output the number of lines in the largest file; no need to output the name of this file. The program must work correctly even if the number of files is not divisible by the number of processes.

### Problem 3 (2 points)
Solve the Distinct Substrings problem at Sphere online judge: http://www.spoj.com/problems/DISUBSTR/. Provide code passing the test of the judge. Explain how your code works and theoretically estimate the complexity of the algorithm (as $O(f(N))$, where $f(N)$ is some function of the length of the input string). 

### Problem 4 (2 points)
Suppose that we want to distribute $N$ personal projects to $N$ students. Assume that each student $(k)_{k=0}^{N-1}$ has a list of his/her preferences for the projects, expressed as a vector $\mathbf r_k$ of integer ranks assigned to each project. Ranks vary between 0 and $N-1$ without repetitions, the **lower** the rank the **more preferable** the project. (For example, the first student's ranks are $\mathbf r_0 = [2,1,0]$, the second's $\mathbf r_1 = [0,2,1]$ and the third $\mathbf r_2 = [0,1,2]$). We want to distribute the projects so as to maximize the total preference, i.e., if $n_k$ denotes the project assigned to the $k$'th student, we want to make $f = \sum_{k=0}^{N-1} \mathbf r_k[n_k]$ as small as possible. (In the example above the optimal distribution is $n_0=2, n_1=0, n_2=1$, which gives $f=1$).  
  * Come up with an algorithm optimizing the distribution and implement it in a Python or C/C++ program. The algorithm should accept the preference vectors and output a recommended distribution $(n_k)_{k=1}^N$. The algorithm need not find the best solution, but is expected to generally produce better solutions than would have been obtained by randomly distributing the projects. The algorithm should be reasonably fast, say run in not more than a few seconds for $N=30$. 
  * Compare experimentally your algorithm with the trivial algorithm producing a random distribution. To this end, perform $M=1000$ experiments in each of which 1) random preference vectors for $N=30$ students and projects are generated; 2) the objective function $f$ is evaluated for both algorithms. After finishing all the experiments, plot the two respective distributions of the obtained $M$ values of $f$ and compute the mean values of $f$ for both algorithms. 
  
### Problem 5 (2 points)
Suppose that we have developed an algorithm that is supposed to generate independent (quasi-)random numbers uniformly distributed in the interval $[0,1]$. To test our algorithm, we perform a series of experiments. In each experiment, we generate $N=10^3$ numbers $(x_n)_{n=1}^N$ with our algorithm, and compute the minimum distance $r=\min_{1 \le n < m\le N}|x_n-x_m|$ between them. We observe that in more than 99% of such experiments we obtain $r < 10^{-5}$. Does this observation contradict the hypothesis of generating independent uniformly distributed random numbers? Explain your answer.

Task 1 

1) 20 times
0.75 % of parallelization, speedup = 1/(1-0.75) = 4

2) speedup = $\frac{1}{(\alpha + \frac{1 - \alpha}{p})}$

speedup = 2;

$\alpha = 0.75$ 

=> p = 3

efficacy = 2/3

Task 2 



In [31]:
import glob

# 1. list all text files in the directory
rel_filepaths = glob.glob("*.txt")

# 2. (optional) create a function to read the number of rows in a file
def count_rows_one(filepath):
  res = 0
  f = open(filepath, 'r')
  res = len(f.readlines())
  f.close()

  return res

def count_rows(filepaths):
  res = 0
  ress = []
  for filepath in filepaths:
    f = open(filepath, 'r')
    res = len(f.readlines())
    ress.append(res)
    f.close()



In [ ]:
len(rel_filepaths)

2

In [1]:
!pip install mpi4py

     |████████████████████████████████| 2.4 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.1-cp37-cp37m-linux_x86_64.whl size=2180584 sha256=d79bfa26fd479411c18287423b565700096e04e71ab8d343abbb9b20d3240fdb
  Stored in directory: /root/.cache/pip/wheels/91/be/c0/2b0347be1de5cd8ca9fe67da7ec8c3fe8930fcb6b0df6f2255
Successfully built mpi4py


In [37]:
!mpirun --allow-run-as-root -n 7 python prog.py 

Max size: 67


In [40]:
import numpy as np

def invPerm(p):
    '''Invert the permutation p'''
    s = np.empty(p.size, p.dtype)
    s[p] = np.arange(p.size)
    return s


def getSA(A):
    if not type(A) is np.ndarray:
        A = np.array(list(A))
    N = len(A) 
    M = int(np.ceil(np.log2(N)))+1   # number of iterations
    
    # auxiliary arrays; row m stores results after m'th step:
    
    # positions of sorted length-(2**m) sequences in A
    P = np.zeros((M,N), dtype=int) 
    
    # rank (0, 1, etc.) of sorted length-(2**m) sequences after sorting
    Q = np.zeros((M,N), dtype=int)     
    
    # rank of sorted length-(2**m) sequences at its starting position in A;
    # padded by 0 on the right
    R = np.zeros((M,3*N), dtype=int) 

    for k in range(M):
        if k == 0:
            P[0] = np.argsort(A)
            Q[0][1:] = np.cumsum(A[P[0]][1:] != A[P[0]][:-1])
            R[0][:N] = Q[0][invPerm(P[0])]
        else:
            offset = 2**(k-1)
            r = np.lexsort((R[k-1, P[k-1]+offset], R[k-1, P[k-1]]))
            P[k] = P[k-1][r]
            # k'th rank increases iff (k-1)'th rank increases at least for one element of the pair    
            Q[k][1:] = np.cumsum(np.logical_or(R[k-1][P[k]][1:] != R[k-1][P[k]][:-1], 
                                          R[k-1][P[k]+offset][1:] != R[k-1][P[k]+offset][:-1]))
            R[k][:N] = Q[k][invPerm(P[k])]
            
            # early stopping if suffixes already fully sorted (max rank is N-1)
            if Q[k][-1] == N-1: 
                break
    
    SA = P[k]
    return SA, P[:k+1], Q[:k+1], R[:k+1] 



def getLCP(SA, R):
    (M, N) = R.shape
    LCP = np.zeros((len(SA)-1,),dtype=int)
    for m in range(M-1)[::-1]:
        t = (R[m][SA[1:]+LCP] == R[m][SA[:-1]+LCP]).astype(int)
        LCP += (2**m)*t
    return LCP

def getNumString(A):
    N = len(A)
    SA, _, _, R = getSA(A)
    LCP = getLCP(SA, R)
    return int(N*(N-1)/2 - sum(LCP) )
def main():
  n = int(input())
  for i in range(n):
    string = input()
    string += '$'
    print(getNumString(string))


if __name__ == '__main__':
    main()          

2
ccccc
5
ABABA
9


In [38]:
getNumString('CCCCC$')

11

In [34]:
A = 'CCCCC$'
N = len(A)

In [35]:
SA, _, _, R = getSA(A)
LCP = getLCP(SA, R)


In [36]:
LCP

array([0, 1, 2, 3, 4])

In [37]:
print ('Sorted suffixes:')
for n in range(len(A)):
    print (A[SA[n]:])

Sorted suffixes:
$
C$
CC$
CCC$
CCCC$
CCCCC$


In [29]:
5*2 - 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])